In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import datetime
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set(font_scale=2.0)

from sklearn.preprocessing import StandardScaler

## Having a look at the data

In [127]:
data=pd.read_csv('lc_loan.csv',parse_dates=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [128]:
data.shape

(887379, 74)

In [129]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m
0,1077501,1296599,5000.0,5000.0,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I need to upgra...,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,NaN,3.0,0.0,13648.0,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-2015,171.62,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1077430,1314167,2500.0,2500.0,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/22/11 > I plan to use t...,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,NaN,Sep-2013,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1077175,1313524,2400.0,2400.0,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,NaN,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,NaN,2.0,0.0,2956.0,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-2014,649.91,NaN,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1076863,1277178,10000.0,10000.0,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > to pay for prop...,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,NaN,10.0,0.0,5598.0,21.0,37.0,f,0.0,0.0,12226.302212,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-2015,357.48,NaN,Jan-2015,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1075358,1311748,3000.0,3000.0,3000.0,60 months,12.69,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 12/21/11 > I plan on combi...,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,f,766.9,766.9,3242.170000,3242.17,2233.10,1009.07,0.00,0.00,0.00,Jan-2016,67.79,Feb-2016,Jan-2016,0.0,NaN,1.0,INDIVIDUAL,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [130]:
pd.value_counts(data.loan_status).to_frame().reset_index()

,index,loan_status
0,Current,601779
1,Fully Paid,207723
2,Charged Off,45248
3,Late (31-120 days),11591
4,Issued,8460
5,In Grace Period,6253
6,Late (16-30 days),2357
7,Does not meet the credit policy. Status:Fully ...,1988
8,Default,1219
9,Does not meet the credit policy. Status:Charge...,761


#### There are 9 distinct entries for loan_status. I will drop the 'Fully Paid' as they are past entries. Next is to provide dummy variable to loan_status - 0 (good) for current and 1 (bad) for the rest.

In [131]:
data = data[data.loan_status != 'Fully Paid']
data = data[data.loan_status != 'Does not meet the credit policy. Status:Fully Paid']

In [132]:
data['rating'] = np.where((data.loan_status != 'Current'), 1, 0)

In [133]:
pd.value_counts(data.rating).to_frame()

,rating
0,601779
1,75889


In [134]:
print ('Bad Loan Ratio: %.2f%%'  % (data.rating.sum()/len(data)*100))

Bad Loan Ratio: 11.20%


#### This is an imbalanced dataset but as per the numbers I have 75,000 bad loans which would be enough for any model to learn.

## Reviewing the variables and editing them

In [135]:
pd.value_counts(data.application_type).to_frame()

,application_type
INDIVIDUAL,677158
JOINT,510


In [136]:
app_type={'INDIVIDUAL':0,'JOINT':1}
data.application_type.replace(app_type,inplace=True)
# Just replaced the column so that no need to delete later.

In [137]:
pd.value_counts(data.application_type).to_frame()

,application_type
0,677158
1,510


In [138]:
pd.value_counts(data.term).to_frame()

,term
36 months,451761
60 months,225907


In [139]:
term={' 36 months':36,' 60 months':60}
data.term.replace(term,inplace=True)

In [140]:
pd.value_counts(data.term).to_frame()

,term
36,451761
60,225907


In [141]:
pd.value_counts(data.grade).to_frame()

,grade
C,192701
B,187720
D,109028
A,108433
E,57399
F,18166
G,4221


In [142]:
grade=data.grade.unique()
grade.sort()
grade

array(['A', 'B', 'C', 'D', 'E', 'F', 'G'], dtype=object)

In [143]:
for x,e in enumerate(grade):
    data.grade.replace(to_replace=e,value=x,inplace=True)

In [144]:
data.grade.unique()

array([2, 1, 5, 3, 0, 4, 6], dtype=int64)

In [145]:
pd.value_counts(data.sub_grade).to_frame()

,sub_grade
B4,40963
C1,40811
B3,40453
C2,40321
C3,39839
C4,39261
B5,36550
B2,35702
B1,34052
A5,33758


In [146]:
sub_grade=data.sub_grade.unique()
sub_grade.sort()
sub_grade

array(['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1',
       'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2',
       'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3',
       'G4', 'G5'], dtype=object)

In [147]:
for x,e in enumerate(sub_grade):
    data.sub_grade.replace(to_replace=e,value=x,inplace=True)

data.sub_grade.unique()

array([13,  9, 14, 26, 10,  6,  8,  7, 11, 16,  4,  5, 23, 17, 28, 18, 19,
        1, 12,  2, 25, 15,  3, 24, 20, 21, 22, 32, 31, 30, 27, 29,  0, 33,
       34], dtype=int64)

In [148]:
emp_len={'n/a':0,'< 1 year':1,'1 year':2,'2 years':3,'3 years':4,'4 years':5,'5 years':6,'6 years':7,'7 years':8,'8 years':9,'9 years':10,'10+ years':11}
data.emp_length.replace(emp_len,inplace=True)
data.emp_length=data.emp_length.replace(np.nan,0)
data.emp_length.unique()

array([ 1.,  2.,  9.,  5.,  4., 11., 10.,  3.,  8.,  7.,  6.,  0.])

In [149]:
pd.value_counts(data.initial_list_status).to_frame()

,initial_list_status
w,371954
f,305714


In [150]:
int_status={'w':0,'f':1}
data.initial_list_status.replace(int_status,inplace=True)

In [151]:
pd.value_counts(data.initial_list_status).to_frame()

,initial_list_status
0,371954
1,305714


In [152]:
pd.value_counts(data.emp_title).to_frame()
# There are around 212K different titles so this column would be dropped later.

,emp_title
Teacher,11875
Manager,9680
Owner,5028
Registered Nurse,4836
RN,4657
Supervisor,4297
Sales,3674
Project Manager,3406
Driver,3175
Office Manager,3022


In [153]:
pd.value_counts(data.pymnt_plan).to_frame()
# This column would also be dropped as it has just 8 'y'entries.

,pymnt_plan
n,677660
y,8


In [154]:
pd.value_counts(data.zip_code).to_frame()
# Dropped as zip_code has no relation to target variable

,zip_code
750xx,7065
945xx,6976
112xx,6974
606xx,6505
300xx,6181
070xx,5618
331xx,5605
100xx,5403
770xx,5250
900xx,5213


In [93]:
pd.value_counts(data.recoveries).to_frame()

,recoveries
0.00,652991
200.00,10
11.70,9
14.28,9
10.00,9
16.50,8
10.40,8
800.00,8
16.80,8
100.00,8


In [95]:
data['recovery'] = np.where((data.recoveries != 0.00), 1, 0)
# Just converting recovery to a binary variable to know if loan was recovered or not.

In [97]:
pd.value_counts(data.recovery).to_frame()

,recovery
0,652991
1,24677


In [98]:
# Some variables can be converted to datetime.
data.issue_d=pd.to_datetime(data.issue_d)

In [99]:
earliest_cr_line=pd.to_datetime(data.earliest_cr_line)
data.earliest_cr_line=earliest_cr_line.dt.year

In [100]:
data.last_pymnt_d=pd.to_datetime(data.last_pymnt_d)
data.next_pymnt_d=pd.to_datetime(data.next_pymnt_d)
data.last_credit_pull_d=pd.to_datetime(data.last_credit_pull_d)

In [101]:
# Dropping the variables mentioned earlier.
data.drop(['id','member_id','desc','loan_status','url', 'title','collection_recovery_fee','recoveries','policy_code','zip_code','emp_title','pymnt_plan'],axis=1,inplace=True)

In [102]:
data.head(10)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,rating,recovery
1,2500.0,2500.0,2500.0,60,15.27,59.83,2,13,1.0,RENT,30000.0,Source Verified,2011-12-01,car,GA,1.00,0.0,1999.0,5.0,NaN,NaN,3.0,0.0,1687.0,9.4,4.0,1,0.00,0.00,1008.71,1008.71,456.46,435.17,0.0,2013-04-01,119.66,NaT,2013-09-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
4,3000.0,3000.0,3000.0,60,12.69,67.79,1,9,2.0,RENT,80000.0,Source Verified,2011-12-01,other,OR,17.94,0.0,1996.0,0.0,38.0,NaN,15.0,0.0,27783.0,53.9,38.0,1,766.90,766.90,3242.17,3242.17,2233.10,1009.07,0.0,2016-01-01,67.79,2016-02-01,2016-01-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
6,7000.0,7000.0,7000.0,60,15.96,170.08,2,14,9.0,RENT,47004.0,Not Verified,2011-12-01,debt_consolidation,NC,23.51,0.0,2005.0,1.0,NaN,NaN,7.0,0.0,17726.0,85.6,11.0,1,1889.15,1889.15,8136.84,8136.84,5110.85,3025.99,0.0,2016-01-01,170.08,2016-02-01,2016-01-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
8,5600.0,5600.0,5600.0,60,21.28,152.39,5,26,5.0,OWN,40000.0,Source Verified,2011-12-01,small_business,CA,5.55,0.0,2004.0,2.0,NaN,NaN,11.0,0.0,5210.0,32.6,13.0,1,0.00,0.00,646.02,646.02,162.02,294.94,0.0,2012-04-01,152.39,NaT,2012-08-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
9,5375.0,5375.0,5350.0,60,12.69,121.45,1,9,1.0,RENT,15000.0,Verified,2011-12-01,other,TX,18.08,0.0,2004.0,0.0,NaN,NaN,2.0,0.0,9279.0,36.5,3.0,1,0.00,0.00,1476.19,1469.34,673.48,533.42,0.0,2012-11-01,121.45,NaT,2013-03-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
12,9000.0,9000.0,9000.0,36,13.49,305.38,2,10,1.0,RENT,30000.0,Source Verified,2011-12-01,debt_consolidation,VA,10.08,0.0,2004.0,1.0,NaN,NaN,4.0,0.0,10452.0,91.7,9.0,1,0.00,0.00,2270.70,2270.70,1256.14,570.26,0.0,2012-07-01,305.38,NaT,2012-11-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
14,10000.0,10000.0,10000.0,36,10.65,325.74,1,6,4.0,RENT,100000.0,Source Verified,2011-12-01,other,CA,7.06,0.0,1991.0,2.0,NaN,NaN,14.0,0.0,11997.0,55.5,29.0,1,0.00,0.00,7471.99,7471.99,5433.47,1393.42,0.0,2013-10-01,325.74,NaT,2014-03-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
21,21000.0,21000.0,21000.0,36,12.42,701.73,1,8,11.0,RENT,105000.0,Verified,2011-12-01,debt_consolidation,FL,13.22,0.0,1983.0,0.0,NaN,NaN,7.0,0.0,32135.0,90.3,38.0,1,0.00,0.00,14025.40,14025.40,10694.96,3330.44,0.0,2013-09-01,701.73,NaT,2016-01-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
24,6000.0,6000.0,6000.0,36,11.71,198.46,1,7,2.0,RENT,76000.0,Not Verified,2011-12-01,major_purchase,CA,2.40,0.0,2001.0,1.0,NaN,NaN,7.0,0.0,5963.0,29.7,7.0,1,0.00,0.00,2050.14,2050.14,1305.58,475.25,0.0,2012-10-01,198.46,NaT,2013-03-01,0.0,NaN,0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
26,15000.0,15000.0,8725.0,36,14.27,514.64,2,11,10.0,RENT,60000.0,Not Verified,2011-12-01,debt_consolidation,NY,15.22,0.0,200

In [104]:
# function to return missing values from each feature
def meta (dataframe):
    metadata = []
    for f in data.columns:
    
        # Counting null values
        null = data[f].isnull().sum()
     
        dtype = data[f].dtype
    
        f_dict = {
            'varname': f,
            'nulls':null,
            'dtype': dtype
        }
        metadata.append(f_dict)

    meta = pd.DataFrame(metadata, columns=['varname','nulls', 'dtype'])
    meta.set_index('varname', inplace=True)
    meta=meta.sort_values(by=['nulls'],ascending=False)
    return meta

In [105]:
meta(data)

,nulls,dtype
varname,,
dti_joint,677160,float64
annual_inc_joint,677158,float64
verification_status_joint,677158,object
il_util,659177,float64
mths_since_rcnt_il,656998,float64
inq_fi,656440,float64
total_cu_tl,656440,float64
all_util,656440,float64
max_bal_bc,656440,float64


#### Variables: dti_joint, annual_inc_joint and verification_status_joint have so many null values as there are only 510 joint loans. Will replace NaN with 0 and 'None' for status.

In [106]:
data.dti_joint=data.dti_joint.replace(np.nan,0)
data.annual_inc_joint=data.annual_inc_joint.replace(np.nan,0)
data.verification_status_joint=data.verification_status_joint.replace(np.nan,'None')

In [107]:
variables1=['open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'inq_fi', 'total_cu_tl', 'inq_last_12m','collections_12_mths_ex_med']

for e in variables1:
    data[e]=data[e].replace(np.nan,0)
    
meta(data)

,nulls,dtype
varname,,
mths_since_last_record,567148,float64
mths_since_last_major_derog,494756,float64
mths_since_last_delinq,337650,float64
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
tot_coll_amt,14599,float64
total_rev_hi_lim,14599,float64
tot_cur_bal,14599,float64
revol_util,316,float64


In [108]:
pd.value_counts(data.mths_since_last_record).unique()

array([1627, 1625, 1624, 1623, 1619, 1615, 1610, 1609, 1607, 1601, 1593,
       1586, 1553, 1530, 1517, 1505, 1496, 1492, 1483, 1473, 1468, 1460,
       1451, 1447, 1442, 1437, 1388, 1367, 1359, 1263, 1253, 1188, 1146,
       1129, 1127, 1121, 1119, 1093, 1090, 1089, 1087, 1077, 1059, 1043,
       1017,  998,  982,  972,  958,  936,  935,  929,  928,  921,  913,
        906,  905,  896,  884,  870,  869,  864,  862,  850,  849,  844,
        841,  839,  834,  833,  826,  820,  817,  815,  812,  809,  803,
        771,  738,  712,  695,  692,  684,  624,  600,  593,  541,  533,
        511,  471,  469,  412,  400,  395,  375,  370,  355,  340,  318,
        303,  286,  283,  281,  279,  234,  227,  210,  189,  156,  151,
        131,  107,   64,   53,    7,    1], dtype=int64)

In [109]:
pd.value_counts(data.mths_since_last_major_derog).unique()

array([2835, 2794, 2788, 2781, 2768, 2762, 2756, 2730, 2674, 2668, 2665,
       2662, 2644, 2625, 2624, 2621, 2616, 2602, 2588, 2585, 2573, 2572,
       2569, 2555, 2553, 2550, 2549, 2543, 2540, 2532, 2530, 2523, 2517,
       2511, 2502, 2496, 2479, 2458, 2454, 2425, 2416, 2389, 2385, 2310,
       2262, 2261, 2259, 2190, 2162, 2145, 2103, 2093, 2088, 2060, 2051,
       1998, 1989, 1968, 1913, 1840, 1683, 1598, 1570, 1545, 1495, 1491,
       1412, 1361, 1340, 1252, 1153, 1080,  839,  613,  599,  504,  497,
        216,  156,  135,  128,  117,  110,   96,   94,   89,   80,   78,
         76,   73,   71,   70,   69,   67,   66,   65,   63,   59,   57,
         56,   54,   52,   48,   46,   43,   42,   41,   39,   38,   37,
         35,   33,   32,   30,   27,   26,   25,   24,   22,   21,   20,
         19,   17,   16,   15,   14,   11,   10,    8,    7,    6,    5,
          4,    3,    2,    1], dtype=int64)

In [110]:
pd.value_counts(data.mths_since_last_delinq).unique()

array([6941, 6940, 6707, 6698, 6695, 6584, 6405, 6401, 6284, 5977, 5961,
       5920, 5796, 5747, 5691, 5622, 5612, 5509, 5441, 5359, 5325, 5310,
       5255, 5248, 4997, 4992, 4961, 4916, 4910, 4821, 4795, 4660, 4637,
       4541, 4459, 4435, 4406, 4274, 4260, 4196, 4168, 4147, 4107, 3975,
       3961, 3455, 3220, 2974, 2964, 2935, 2919, 2910, 2905, 2900, 2895,
       2889, 2844, 2830, 2828, 2827, 2820, 2810, 2803, 2778, 2775, 2768,
       2693, 2662, 2599, 2528, 2518, 2438, 2350, 2318, 2160, 2036, 2011,
       1931, 1841, 1711,  991,  808,  116,   47,   46,   33,   28,   27,
         23,   20,   19,   17,   16,   14,   13,   10,    9,    8,    7,
          6,    5,    4,    3,    2,    1], dtype=int64)

#### Null values in these columns can't be replaced with 0 as it would mean that the last occurence was very recent. My understanding of these variables is that the key information is whether the specific action took place (delinquency, public record, worse rating), so I will turn these into binary categories of Yes (1), No (0).

In [111]:
data.loc[(data.mths_since_last_delinq.notnull()),'delinq']=1
data.loc[(data.mths_since_last_delinq.isnull()),'delinq']=0

data.loc[(data.mths_since_last_major_derog.notnull()),'derog']=1
data.loc[(data.mths_since_last_major_derog.isnull()),'derog']=0

data.loc[(data.mths_since_last_record.notnull()),'public_record']=1
data.loc[(data.mths_since_last_record.isnull()),'public_record']=0

data.drop(['mths_since_last_delinq','mths_since_last_major_derog','mths_since_last_record'],axis=1,inplace=True)

meta(data)

,nulls,dtype
varname,,
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
total_rev_hi_lim,14599,float64
tot_coll_amt,14599,float64
tot_cur_bal,14599,float64
revol_util,316,float64
last_credit_pull_d,36,datetime64[ns]
inq_last_6mths,3,float64
acc_now_delinq,3,float64


In [112]:
variables2=['tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim']

for e in variables2:
    data[e]=data[e].replace(np.nan,0)
    
meta(data)

,nulls,dtype
varname,,
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
revol_util,316,float64
last_credit_pull_d,36,datetime64[ns]
pub_rec,3,float64
delinq_2yrs,3,float64
acc_now_delinq,3,float64
total_acc,3,float64
open_acc,3,float64


In [113]:
data.revol_util=data.revol_util.replace(np.nan,0)
    
meta(data)

,nulls,dtype
varname,,
next_pymnt_d,45248,datetime64[ns]
last_pymnt_d,17659,datetime64[ns]
last_credit_pull_d,36,datetime64[ns]
open_acc,3,float64
delinq_2yrs,3,float64
acc_now_delinq,3,float64
total_acc,3,float64
pub_rec,3,float64
earliest_cr_line,3,float64


#### Variables last_credit_pull_d is the most recent month LC pulled credit for this loan, issue_d is the date loan was issued and next_payment_d is the date of next payment. There are not insightful variables so will be dropped.

In [114]:
# Binary category if the payment is received or not.
data.loc[(data.last_pymnt_d.notnull()),'pymnt_received']=1
data.loc[(data.last_pymnt_d.isnull()),'pymnt_received']=0

In [115]:
data.drop(['last_pymnt_d','issue_d','last_credit_pull_d','next_pymnt_d'],axis=1,inplace=True)

meta(data)

,nulls,dtype
varname,,
earliest_cr_line,3,float64
inq_last_6mths,3,float64
delinq_2yrs,3,float64
pub_rec,3,float64
total_acc,3,float64
open_acc,3,float64
acc_now_delinq,3,float64
open_acc_6m,0,float64
open_il_6m,0,float64


In [116]:
# After carefully dealing with null values we just have 3 missing values so we replace them by mode.
variables3=['acc_now_delinq', 'open_acc', 'total_acc','pub_rec','delinq_2yrs','inq_last_6mths','earliest_cr_line']

for e in variables3:
    data[e]=data[e].replace(np.nan,data[e].mode()[0])
    
meta(data)

,nulls,dtype
varname,,
loan_amnt,0,float64
last_pymnt_amnt,0,float64
application_type,0,int64
annual_inc_joint,0,float64
dti_joint,0,float64
verification_status_joint,0,object
acc_now_delinq,0,float64
tot_coll_amt,0,float64
tot_cur_bal,0,float64


In [117]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,rating,recovery,delinq,derog,public_record,pymnt_received
1,2500.0,2500.0,2500.0,60,15.27,59.83,2,13,1.0,RENT,30000.0,Source Verified,car,GA,1.00,0.0,1999.0,5.0,3.0,0.0,1687.0,9.4,4.0,1,0.00,0.00,1008.71,1008.71,456.46,435.17,0.0,119.66,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0,1.0
4,3000.0,3000.0,3000.0,60,12.69,67.79,1,9,2.0,RENT,80000.0,Source Verified,other,OR,17.94,0.0,1996.0,0.0,15.0,0.0,27783.0,53.9,38.0,1,766.90,766.90,3242.17,3242.17,2233.10,1009.07,0.0,67.79,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1.0,0.0,0.0,1.0
6,7000.0,7000.0,7000.0,60,15.96,170.08,2,14,9.0,RENT,47004.0,Not Verified,debt_consolidation,NC,23.51,0.0,2005.0,1.0,7.0,0.0,17726.0,85.6,11.0,1,1889.15,1889.15,8136.84,8136.84,5110.85,3025.99,0.0,170.08,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,1.0
8,5600.0,5600.0,5600.0,60,21.28,152.39,5,26,5.0,OWN,40000.0,Source Verified,small_business,CA,5.55,0.0,2004.0,2.0,11.0,0.0,5210.0,32.6,13.0,1,0.00,0.00,646.02,646.02,162.02,294.94,0.0,152.39,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0,1.0
9,5375.0,5375.0,5350.0,60,12.69,121.45,1,9,1.0,RENT,15000.0,Verified,other,TX,18.08,0.0,2004.0,0.0,2.0,0.0,9279.0,36.5,3.0,1,0.00,0.00,1476.19,1469.34,673.48,533.42,0.0,121.45,0.0,0,0.0,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,0.0,0.0,0.0,1.0


## Prepare the data for modeling

In [118]:
# X is the df for input features
X=data.drop(['rating'],axis=1,inplace=False)
y=data.rating

In [120]:
num_cols = X.columns[X.dtypes.apply(lambda c: np.issubdtype(c, np.number))]
num_cols

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'sub_grade', 'emp_length', 'annual_inc', 'dti',
       'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'open_acc',
       'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'last_pymnt_amnt', 'collections_12_mths_ex_med',
       'application_type', 'annual_inc_joint', 'dti_joint', 'acc_now_delinq',
       'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_il_6m',
       'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il',
       'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'recovery',
       'delinq', 'derog', 'public_record', 'pymnt_received'],
      dtype='object')

In [121]:
# data is in different ranges so we scale it.
scaler=StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])
X.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,recovery,delinq,derog,public_record,pymnt_received
1,-1.494608,-1.494530,-1.492084,1.414131,0.460170,-1.578013,0.122656,0.282623,-1.447841,RENT,-0.683244,Source Verified,car,GA,-0.928551,-0.374242,0.163975,4.583384,-1.611536,-0.341277,-0.678975,-1.969564,-1.796942,1.103029,-1.356000,-1.356001,-0.764285,-0.763924,-0.775548,-0.602142,-0.097547,-0.733340,-0.115772,-0.027444,-0.02476,-0.025503,None,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,5.144076,-1.003500,-0.608031,-0.441441,0.163572
4,-1.435786,-1.435688,-1.433254,1.414131,-0.123573,-1.545348,-0.637278,-0.331601,-1.192859,RENT,0.070865,Source Verified,other,OR,-0.044239,-0.374242,-0.232145,-0.679328,0.599602,-0.341277,0.447634,-0.073554,1.072696,1.103029,-1.261492,-1.261455,-0.358170,-0.357129,-0.314057,-0.330915,-0.097547,-0.850012,-0.115772,-0.027444,-0.02476,-0.025503,None,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,-0.608031,-0.441441,0.163572
6,-0.965208,-0.964953,-0.962612,1.414131,0.616287,-1.125590,0.122656,0.436178,0.592018,RENT,-0.426787,Not Verified,debt_consolidation,NC,0.246529,-0.374242,0.956216,0.373214,-0.874490,-0.341277,0.013456,1.277087,-1.206134,1.103029,-1.123194,-1.123100,0.531838,0.534369,0.433454,0.622287,-0.097547,-0.619929,-0.115772,-0.027444,-0.02476,-0.025503,None,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,-0.194398,-1.003500,-0.608031,-0.441441,0.163572
8,-1.129910,-1.129710,-1.127337,1.414131,1.819974,-1.198183,2.402458,2.278849,-0.427912,OWN,-0.532422,Source Verified,small_business,CA,-0.691029,-0.374242,0.824175,1.425757,-0.137444,-0.341277,-0.526881,-0.981082,-1.037332,1.103029,-1.356000,-1.356001,-0.830234,-0.829983,-0.852031,-0.668415,-0.097547,-0.659720,-0.115772,-0.027444,-0.02476,-0.025503,None,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,5.144076,-1.003500,-0.608031,-0.441441,0.163572
9,-1.156380,-1.156189,-1.156752,1.414131,-0.123573,-1.325149,-0.637278,-0.331601,-1.447841,RENT,-0.909477,Verified,other,TX,-0.036931,-0.374242,0.824175,-0.679328,-1.795797,-0.341277,-0.351215,-0.814915,-1.881343,1.103029,-1.356000,-1.356001,-0.679282,-0.680027,-0.719176,-0.555709,-0.097547,-0.729314,-0.115772,-0.027444,-0.02476,-0.025503,None,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,5.144076,-1.003500,-0.608031,-0.441441,0.163572


In [122]:
# Get numerical values for all remaining categorical features.
X=pd.get_dummies(X,drop_first=True)
X.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,recovery,delinq,derog,public_record,pymnt_received,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified
1,-1.494608,-1.494530,-1.492084,1.414131,0.460170,-1.578013,0.122656,0.282623,-1.447841,-0.683244,-0.928551,-0.374242,0.163975,4.583384,-1.611536,-0.341277,-0.678975,-1.969564,-1.796942,1.103029,-1.356000,-1.356001,-0.764285,-0.763924,-0.775548,-0.602142,-0.097547,-0.733340,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,5.144076,-1.003500,-0.608031,-0.441441,0.163572,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-1.435786,-1.435688,-1.433254,1.414131,-0.123573,-1.545348,-0.637278,-0.331601,-1.192859,0.070865,-0.044239,-0.374242,-0.232145,-0.679328,0.599602,-0.341277,0.447634,-0.073554,1.072696,1.103029,-1.261492,-1.261455,-0.358170,-0.357129,-0.314057,-0.330915,-0.097547,-0.850012,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,-0.608031,-0.441441,0.163572,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,-0.965208,-0.964953,-0.962612,1.414131,0.616287,-1.125590,0.122656,0.436178,0.592018,-0.426787,0.246529,-0.374242,0.956216,0.373214,-0.874490,-0.341277,0.013456,1.277087,-1.206134,1.103029,-1.123194,-1.123100,0.531838,0.534369,0.433454,0.622287,-0.097547,-0.619929,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.1121,-0.887578,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.817419,-0.097143,-0.087407,-0.100916,-0.194398,-1.003500,-0.608031,-0.441441,0.163572,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,-1.129910,-1.129710,-1.127337,1.414131,1

In [123]:
X.shape

(677668, 128)

In [184]:
from sklearn.model_selection import cross_val_score, cross_val_predict, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report,accuracy_score 

In [155]:
# As the dataset is imbalanced (11% is bad loan) we try both oversampling and undersampling

# 1. Oversampling

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

# Combine them again for resampling
train_data = pd.concat([X_train, y_train], axis=1)
train_data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,recovery,delinq,derog,public_record,pymnt_received,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified,rating
682726,0.646521,0.647316,0.649337,1.414131,-0.214076,0.078489,0.122656,-0.178045,1.101982,-0.457011,-0.345970,0.739052,-0.496226,-0.679328,0.968125,-0.341277,-0.450726,-2.012171,0.903894,-0.906594,1.004138,1.005097,-0.445801,-0.444906,-0.491976,-0.234968,-0.097547,0.025155,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.112100,-0.779537,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,1.317526,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,-0.608031,-0.441441,0.163572,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
310776,-0.612275,-0.611901,-0.609630,-0.707148,-0.168824,-0.450917,-0.637278,-0.331601,-0.937876,-0.532422,0.342582,2.965638,-0.100105,-0.679328,-0.505967,-0.341277,-0.589955,1.524208,0.481888,1.103029,-1.356000,-1.356001,-0.461131,-0.460262,-0.426653,-0.393663,-0.097547,-0.250117,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.112100,0.034667,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.711831,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,1.644654,-0.441441,0.163572,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
532253,-0.376986,-0.376533,-0.374309,-0.707148,-1.582930,-0.320094,-1.397212,-1.560048,1.101982,0.085947,-0.288547,-0.374242,-1.024386,-0.679328,-0.874490,-0.341277,-0.399093,-1.394370,-0.362123,-0.906594,0.047682,0.048252,-0.815222,-0.814946,-0.735790,-0.751537,-0.097547,-0.178409,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.112100,0.374370,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,0.101972,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,-0.608031,-0.441441,0.163572,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
687107,0

In [156]:
pd.value_counts(train_data.rating) #imbalanced

0    421203
1     53164
Name: rating, dtype: int64

In [165]:
bad_loan = train_data[train_data.rating==1]
good_loan = train_data[train_data.rating==0]

bad_upsampled = resample(bad_loan,
 replace=True, # sample with replacement
 n_samples=len(good_loan), # match number in majority class
 random_state=27)

In [166]:
upsampled = pd.concat([good_loan, bad_upsampled])

In [167]:
upsampled.rating.value_counts()

1    421203
0    421203
Name: rating, dtype: int64

In [168]:
upsampled.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,last_pymnt_amnt,collections_12_mths_ex_med,application_type,annual_inc_joint,dti_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,recovery,delinq,derog,public_record,pymnt_received,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,verification_status_joint_Not Verified,verification_status_joint_Source Verified,verification_status_joint_Verified,rating
682726,0.646521,0.647316,0.649337,1.414131,-0.214076,0.078489,0.122656,-0.178045,1.101982,-0.457011,-0.345970,0.739052,-0.496226,-0.679328,0.968125,-0.341277,-0.450726,-2.012171,0.903894,-0.906594,1.004138,1.005097,-0.445801,-0.444906,-0.491976,-0.234968,-0.097547,0.025155,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.112100,-0.779537,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,1.317526,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,-0.608031,-0.441441,0.163572,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
532253,-0.376986,-0.376533,-0.374309,-0.707148,-1.582930,-0.320094,-1.397212,-1.560048,1.101982,0.085947,-0.288547,-0.374242,-1.024386,-0.679328,-0.874490,-0.341277,-0.399093,-1.394370,-0.362123,-0.906594,0.047682,0.048252,-0.815222,-0.814946,-0.735790,-0.751537,-0.097547,-0.178409,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.112100,0.374370,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,0.101972,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,-0.608031,-0.441441,0.163572,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
687107,0.328881,0.329570,0.331653,-0.707148,-0.386031,0.613312,-0.637278,-0.331601,1.101982,-0.532422,0.566531,-0.374242,0.956216,-0.679328,-0.137444,-0.341277,-0.267850,-0.559273,-0.277722,-0.906594,0.543423,0.544195,-0.301933,-0.300798,-0.222202,-0.351871,-0.097547,0.333223,-0.115772,-0.027444,-0.02476,-0.025503,-0.068012,-0.112100,-0.632335,-0.118609,-0.12266,-0.10818,-0.125659,-0.107049,-0.11518,-0.159221,-0.120199,-0.13373,-0.132845,-0.170558,-0.137533,-0.097143,-0.087407,-0.100916,-0.194398,0.996512,1.644654,-0.441441,0.163572,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
701765,-1.20

In [169]:
X_up=upsampled.drop('rating',axis=1,inplace=False)
y_up=upsampled.rating

In [170]:
# Splitting again after resampling
X_up_train, X_up_test, y_up_train, y_up_test = train_test_split( X_up, y_up, test_size=0.3, random_state=42)

In [207]:
# First model
rf = RandomForestClassifier(n_estimators=30, criterion='gini', max_depth=20)
rf.fit(X_up_train, y_up_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [209]:
print('Metrics for the Random Forest Classifier after Oversampling')
print('\n',classification_report(y_up_test, rf.predict(X_up_test)))
print('ROC AUC:', roc_auc_score(y_up_test, rf.predict(X_up_test)))

Metrics for the Random Forest Classifier after Oversampling

               precision    recall  f1-score   support

           0       0.86      0.98      0.92    126325
           1       0.98      0.84      0.90    126397

   micro avg       0.91      0.91      0.91    252722
   macro avg       0.92      0.91      0.91    252722
weighted avg       0.92      0.91      0.91    252722

ROC AUC: 0.9112229018730093


In [189]:
# Second Model
lr=LogisticRegression()
lr.fit(X_up_train, y_up_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [210]:
print('Metrics for the Logistic Regression after Oversampling')
print('\n',classification_report(y_up_test, lr.predict(X_up_test)))
print('ROC AUC:', roc_auc_score(y_up_test, lr.predict(X_up_test)))

Metrics for the Logistic Regression after Oversampling

               precision    recall  f1-score   support

           0       0.80      0.97      0.88    126325
           1       0.96      0.77      0.85    126397

   micro avg       0.87      0.87      0.87    252722
   macro avg       0.88      0.87      0.86    252722
weighted avg       0.88      0.87      0.86    252722

ROC AUC: 0.8656751490679567


### Random Forest has been tuned with a 10 fold cross validation and because it is an ensemble is the reason it does better.

In [212]:
# 2. Undersampling

good_downsampled = resample(good_loan,
 replace=True, # sample with replacement
 n_samples=len(bad_loan), # match number in majority class
 random_state=27)

downsampled = pd.concat([bad_loan, good_downsampled])

In [213]:
downsampled.rating.value_counts()

1    53164
0    53164
Name: rating, dtype: int64

In [214]:
X_down=downsampled.drop('rating',axis=1,inplace=False)
y_down=downsampled.rating

X_down_train, X_down_test, y_down_train, y_down_test = train_test_split( X_down, y_down, test_size=0.3, random_state=42)

In [215]:
# Model 1

rf.fit(X_down_train, y_down_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [216]:
print('Metrics for the Random Forest Classifier after Downsampling')
print('\n',classification_report(y_down_test, rf.predict(X_down_test)))
print('ROC AUC:', roc_auc_score(y_down_test, rf.predict(X_down_test)))

Metrics for the Random Forest Classifier after Downsampling

               precision    recall  f1-score   support

           0       0.80      0.97      0.88     15885
           1       0.96      0.76      0.85     16014

   micro avg       0.87      0.87      0.87     31899
   macro avg       0.88      0.87      0.87     31899
weighted avg       0.88      0.87      0.87     31899

ROC AUC: 0.8673473977502923


In [223]:
parameters = {
    'n_estimators'      : [50,60,70],
    'max_depth'         : [30,40,50],
    'criterion'         :  ['gini']
}

clf = GridSearchCV(rf, parameters, cv=10, n_jobs=-1)

In [224]:
clf.fit(X_down_train, y_down_train)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [50, 60, 70], 'max_depth': [30, 40, 50], 'criterion': ['gini']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [225]:
print(clf.best_params_)

{'criterion': 'gini', 'max_depth': 30, 'n_estimators': 60}


In [226]:
rf_final=RandomForestClassifier(max_depth=30, criterion='gini',n_estimators=60)
rf_final.fit(X_down_train, y_down_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=60, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [227]:
print('Metrics for the Random Forest Classifier after Downsampling, parameter tuning and 10 fold CV')
print('\n',classification_report(y_down_test, rf_final.predict(X_down_test)))
print('ROC AUC:', roc_auc_score(y_down_test, rf_final.predict(X_down_test)))

Metrics for the Random Forest Classifier after Downsampling, parameter tuning and 10 fold CV

               precision    recall  f1-score   support

           0       0.81      0.97      0.88     15885
           1       0.96      0.77      0.85     16014

   micro avg       0.87      0.87      0.87     31899
   macro avg       0.88      0.87      0.87     31899
weighted avg       0.88      0.87      0.87     31899

ROC AUC: 0.8683681110944825


In [228]:
# Model 2

lr.fit(X_down_train, y_down_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [229]:
print('Metrics for the Logistic Regression after Undersampling')
print('\n',classification_report(y_down_test, lr.predict(X_down_test)))
print('ROC AUC:', roc_auc_score(y_down_test, lr.predict(X_down_test)))

Metrics for the Logistic Regression after Undersampling

               precision    recall  f1-score   support

           0       0.80      0.96      0.87     15885
           1       0.95      0.76      0.85     16014

   micro avg       0.86      0.86      0.86     31899
   macro avg       0.88      0.86      0.86     31899
weighted avg       0.88      0.86      0.86     31899

ROC AUC: 0.8628424927527413
